In [43]:
import cv2
import json
import numpy as np
import pandas as pd
import youtube_dl

from youtube_dl import DownloadError

from sklearn.model_selection import train_test_split

In [4]:
vids_df = pd.read_csv('../data/yt8m_indoor_outdoor_labels_id.csv', index_col=0)
vids_df

,labels,target,yt8m_id,yt_id
0,"[88, 184, 420, 514]",indoors,k5a7,LLf2i_EK6RU
1,"[15, 18, 60, 87, 545, 1696]",outdoors,Nda7,bkK-qbwAgJo
2,"[511, 514, 1148, 2277, 2405]",indoors,Dca7,jsFiMne0Eyk
3,[60],outdoors,OEa7,p4Ub6lYdK38
4,"[11, 20, 22, 29, 176, 307, 2561]",indoors,EMa7,KW2WY3GIpt4
...,...,...,...,...
2891,"[11, 262, 370]",outdoors,sa9Y,RF_c2Zp5AhU
2892,"[184, 191, 985]",indoors,jD9Y,3GHSzGIqgTg
2893,"[60, 210, 685]",outdoors,lV9Y,xK2AdjZBNoA
2894,"[60, 2701]",outdoors,F99Y,6w6VXsszoak


### Using Youtube_DL library
Lets see what the info looks like

In [8]:
ydl_options = {}
ydl = youtube_dl.YoutubeDL(ydl_options)
yt_info = ydl.extract_info('https://www.youtube.com/watch?v=LLf2i_EK6RU', download=False)
print(yt_info)

[youtube] LLf2i_EK6RU: Downloading webpage
[youtube] LLf2i_EK6RU: Downloading video info webpage
{'id': 'LLf2i_EK6RU', 'uploader': 'Canal Casa Sul', 'uploader_id': 'ProgramaCasaSul', 'uploader_url': 'http://www.youtube.com/user/ProgramaCasaSul', 'channel_id': 'UCdEC-C0_i1-GJTfrpkEv_qw', 'channel_url': 'http://www.youtube.com/channel/UCdEC-C0_i1-GJTfrpkEv_qw', 'upload_date': '20110829', 'license': None, 'creator': 'Spyro Gyra', 'title': 'IDEIAS PARA ILUMINAR QUARTO E BANHEIRO DE UM JEITO ORIGINAL', 'alt_title': "Not for Nothin'", 'thumbnail': 'https://i.ytimg.com/vi/LLf2i_EK6RU/hqdefault.jpg', 'description': 'Ambientes distintos com um toque em comum: a iluminação. Veja o que foi feito em um quarto de moça e um banheiro público a partir de dicas da GREY HOUSE, na Casacor Paraná 2011 e que continuam atuais', 'categories': ['Howto & Style'], 'tags': ['iluminação', 'decoração de quarto', 'decoração de banheiros', 'greyhouse', 'marcelo lopes', 'rolim de moura', 'projeto luminotécnico', 'ilu

In [9]:
print(yt_info.keys())

dict_keys(['id', 'uploader', 'uploader_id', 'uploader_url', 'channel_id', 'channel_url', 'upload_date', 'license', 'creator', 'title', 'alt_title', 'thumbnail', 'description', 'categories', 'tags', 'subtitles', 'automatic_captions', 'duration', 'age_limit', 'annotations', 'chapters', 'webpage_url', 'view_count', 'like_count', 'dislike_count', 'average_rating', 'formats', 'is_live', 'start_time', 'end_time', 'series', 'season_number', 'episode_number', 'track', 'artist', 'album', 'release_date', 'release_year', 'extractor', 'webpage_url_basename', 'extractor_key', 'playlist', 'playlist_index', 'thumbnails', 'display_id', 'requested_subtitles', 'requested_formats', 'format', 'format_id', 'width', 'height', 'resolution', 'fps', 'vcodec', 'vbr', 'stretched_ratio', 'acodec', 'abr', 'ext'])


In [18]:
formats=yt_info.get('formats', None)
format_notes = [x['format_note'] for x in formats]
print(format_notes)
print(formats[7])

['tiny', 'tiny', 'tiny', 'tiny', '144p', '144p', '240p', '240p', '360p', '360p', '480p', '480p', '360p', '360p']
{'format_id': '133', 'url': 'https://r6---sn-qxoedn7e.googlevideo.com/videoplayback?expire=1573804249&ei=eQTOXbCJNruAir4Pi7eD6Ac&ip=71.218.115.0&id=o-ALaQ5w4MUHO0MjjdOwfz3LP7l4cflSgCnKJChDLl51i3&itag=133&aitags=133%2C134%2C135%2C160%2C242%2C243%2C244%2C278&source=youtube&requiressl=yes&mm=31%2C29&mn=sn-qxoedn7e%2Csn-qxo7rn7e&ms=au%2Crdu&mv=m&mvi=5&pl=15&initcwndbps=862500&mime=video%2Fmp4&gir=yes&clen=6997709&dur=228.795&lmt=1377574979076269&mt=1573782541&fvip=6&keepalive=yes&fexp=23842630&c=WEB&sparams=expire%2Cei%2Cip%2Cid%2Caitags%2Csource%2Crequiressl%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&lsparams=mm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AHylml4wRAIgJAz0pIscDE31pszTIzBEiOg7wzKnTnyhx6wNydutaHQCID-sFGFbsY9gFGrGk37s1OXS9qLsiRrwl5dnc9jH50CJ&sig=ALgxI2wwRgIhAPMILEVrJ7I3oSynQW56CrFi1H2SA_0j8R-w48_bYHleAiEAqjBQ8koCi0r5UiDwMdJAyya6Zh4NwBsaqsASN_oUP4Q=&ratebypass=yes', 'player_u

### Lets make a function that pulls a set number of frames
For now, we don't want every frame of the video
Lets lean towards more videos with a few frames over a lot of frames from a few videos  

Need to deal with videos that are private or not available in region  (DownloadError)

#### Issue 
For some reason, when iterating through the videos, one video went way past the number of frame, so we're going to cut it off after it gets frames_wanted. A place to investigate

In [44]:
def get_frames(yt_id, output_dir, frames_wanted, format_wanted):
    yt_url = f"https://www.youtube.com/watch?v={yt_id}"
    ydl_options = {}
    
    ydl = youtube_dl.YoutubeDL(ydl_options)
    
    try: 
        yt_info = ydl.extract_info(yt_url, download=False)
    except DownloadError:
        print("Video not available, DownloadError")
        return
    
    formats = yt_info.get('formats', None)
    
    for f in formats:
        if f.get('format_note') == format_wanted:
            url = f.get('url', None)
            
            cap = cv2.VideoCapture(url)
            
            if not cap.isOpened():
                print('Video not opened')
                
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            
            print(f"Total Frame Count: {total_frames}")
            print(f"FPS: {fps}")
            
            frame_iter = int(total_frames/(frames_wanted)+1)
            
            frame_num = 0
            
            for _ in range(frames_wanted):
                frame_num += frame_iter
                
                cap.set(1, frame_num-1)
                ret, frame = cap.read()
                
                if not ret:
                    break
                
                output_image = output_dir + f"/{yt_id}_{frame_num:06}.jpg"
                
                cv2.imwrite(output_image, frame)
                
                print(f"Writing Frame to: {output_image}")
            
            cap.release()
                
            break # Can have multiple of requested format, so get only 1

### Test function
Have to break after getting the asked for format_note because there can be multiples of each

In [37]:
get_frames('LLf2i_EK6RU', '../data/frames/fxn_test', 10, '240p')

[youtube] LLf2i_EK6RU: Downloading webpage
[youtube] LLf2i_EK6RU: Downloading video info webpage
Total Frame Count: 6857
FPS: 29
Writing Frame to: ../data/frames/fxn_test/LLf2i_EK6RU_000686.jpg
Writing Frame to: ../data/frames/fxn_test/LLf2i_EK6RU_001372.jpg
Writing Frame to: ../data/frames/fxn_test/LLf2i_EK6RU_002058.jpg
Writing Frame to: ../data/frames/fxn_test/LLf2i_EK6RU_002744.jpg
Writing Frame to: ../data/frames/fxn_test/LLf2i_EK6RU_003430.jpg
Writing Frame to: ../data/frames/fxn_test/LLf2i_EK6RU_004116.jpg
Writing Frame to: ../data/frames/fxn_test/LLf2i_EK6RU_004802.jpg
Writing Frame to: ../data/frames/fxn_test/LLf2i_EK6RU_005488.jpg
Writing Frame to: ../data/frames/fxn_test/LLf2i_EK6RU_006174.jpg


### Start parsing through our list of videos

Currently about 25% indoors, 75% outdoors  
If we wanted, we could downsample outdoors to match, upsample indoors, or try to get more indoor vids  
To increase all numbers, we could pull a larger number of shards from the original dataset  

Instructions asked for a balanced dataset, so we will do a random selection of 705 outdoors vids for now

In [38]:
vids_df.groupby('target').count()

,labels,yt8m_id,yt_id
target,,,
indoors,705,705,705
outdoors,2191,2191,2191


In [39]:
outdoors_df = vids_df[vids_df['target']=='outdoors'].sample(n=705)

In [40]:
final_df = vids_df[vids_df['target']=='indoors'].append(outdoors_df)
final_df.groupby('target').count()

,labels,yt8m_id,yt_id
target,,,
indoors,705,705,705
outdoors,705,705,705


### Train Test Split
We could train/test split AFTER we acquire our frames, but it might be a little more robust to split on the videos

In [41]:
train_df, test_df = train_test_split(final_df, test_size=0.2, random_state=42)

### Iterate through our list of videos to get frames
Split into train/test folders, and then further folders on the target  
Since we have ~700 videos, lets get 10 frames per vid

In [45]:
for _, row in train_df.iterrows():
    output_dir = f"../data/frames/train/{row['target']}"
    get_frames(row['yt_id'], output_dir, 10, '240p')

[youtube] D3RppgcXi00: Downloading webpage
[youtube] D3RppgcXi00: Downloading video info webpage
Total Frame Count: 4072
FPS: 29
Writing Frame to: ../data/frames/train/outdoors/D3RppgcXi00_000408.jpg
Writing Frame to: ../data/frames/train/outdoors/D3RppgcXi00_000816.jpg
Writing Frame to: ../data/frames/train/outdoors/D3RppgcXi00_001224.jpg
Writing Frame to: ../data/frames/train/outdoors/D3RppgcXi00_001632.jpg
Writing Frame to: ../data/frames/train/outdoors/D3RppgcXi00_002040.jpg
Writing Frame to: ../data/frames/train/outdoors/D3RppgcXi00_002448.jpg
Writing Frame to: ../data/frames/train/outdoors/D3RppgcXi00_002856.jpg
Writing Frame to: ../data/frames/train/outdoors/D3RppgcXi00_003264.jpg
Writing Frame to: ../data/frames/train/outdoors/D3RppgcXi00_003672.jpg
[youtube] ovmANDtIrj0: Downloading webpage
[youtube] ovmANDtIrj0: Downloading video info webpage
[youtube] ovmANDtIrj0: Downloading MPD manifest
[youtube] xEPQ8E3GB9E: Downloading webpage
[youtube] xEPQ8E3GB9E: Downloading video inf

Writing Frame to: ../data/frames/train/outdoors/zjhwjy7I32g_003120.jpg
Writing Frame to: ../data/frames/train/outdoors/zjhwjy7I32g_003640.jpg
Writing Frame to: ../data/frames/train/outdoors/zjhwjy7I32g_004160.jpg
Writing Frame to: ../data/frames/train/outdoors/zjhwjy7I32g_004680.jpg
Writing Frame to: ../data/frames/train/outdoors/zjhwjy7I32g_005200.jpg
[youtube] ywaRNE38Kvw: Downloading webpage
[youtube] ywaRNE38Kvw: Downloading video info webpage
Total Frame Count: 12287
FPS: 29
Writing Frame to: ../data/frames/train/indoors/ywaRNE38Kvw_001229.jpg
Writing Frame to: ../data/frames/train/indoors/ywaRNE38Kvw_002458.jpg
Writing Frame to: ../data/frames/train/indoors/ywaRNE38Kvw_003687.jpg
Writing Frame to: ../data/frames/train/indoors/ywaRNE38Kvw_004916.jpg
Writing Frame to: ../data/frames/train/indoors/ywaRNE38Kvw_006145.jpg
Writing Frame to: ../data/frames/train/indoors/ywaRNE38Kvw_007374.jpg
Writing Frame to: ../data/frames/train/indoors/ywaRNE38Kvw_008603.jpg
Writing Frame to: ../data

Writing Frame to: ../data/frames/train/indoors/Gge38uuFo5o_001018.jpg
Writing Frame to: ../data/frames/train/indoors/Gge38uuFo5o_002036.jpg
Writing Frame to: ../data/frames/train/indoors/Gge38uuFo5o_003054.jpg
Writing Frame to: ../data/frames/train/indoors/Gge38uuFo5o_004072.jpg
Writing Frame to: ../data/frames/train/indoors/Gge38uuFo5o_005090.jpg
Writing Frame to: ../data/frames/train/indoors/Gge38uuFo5o_006108.jpg
Writing Frame to: ../data/frames/train/indoors/Gge38uuFo5o_007126.jpg
Writing Frame to: ../data/frames/train/indoors/Gge38uuFo5o_008144.jpg
Writing Frame to: ../data/frames/train/indoors/Gge38uuFo5o_009162.jpg
[youtube] -_dGgSJz_i0: Downloading webpage
[youtube] -_dGgSJz_i0: Downloading video info webpage
Total Frame Count: 5559
FPS: 25
Writing Frame to: ../data/frames/train/outdoors/-_dGgSJz_i0_000556.jpg
Writing Frame to: ../data/frames/train/outdoors/-_dGgSJz_i0_001112.jpg
Writing Frame to: ../data/frames/train/outdoors/-_dGgSJz_i0_001668.jpg
Writing Frame to: ../data/fr

ERROR: This video contains content from eTSURIBITO. It is not available in your country.
Sorry about that.


Video not available, DownloadError
[youtube] KD1PWPM4E4o: Downloading webpage
[youtube] KD1PWPM4E4o: Downloading video info webpage
Total Frame Count: 4715
FPS: 29
Writing Frame to: ../data/frames/train/outdoors/KD1PWPM4E4o_000472.jpg
Writing Frame to: ../data/frames/train/outdoors/KD1PWPM4E4o_000944.jpg
Writing Frame to: ../data/frames/train/outdoors/KD1PWPM4E4o_001416.jpg
Writing Frame to: ../data/frames/train/outdoors/KD1PWPM4E4o_001888.jpg
Writing Frame to: ../data/frames/train/outdoors/KD1PWPM4E4o_002360.jpg
Writing Frame to: ../data/frames/train/outdoors/KD1PWPM4E4o_002832.jpg
Writing Frame to: ../data/frames/train/outdoors/KD1PWPM4E4o_003304.jpg
Writing Frame to: ../data/frames/train/outdoors/KD1PWPM4E4o_003776.jpg
Writing Frame to: ../data/frames/train/outdoors/KD1PWPM4E4o_004248.jpg
[youtube] iPJ86uecLD0: Downloading webpage
[youtube] iPJ86uecLD0: Downloading video info webpage
Total Frame Count: 6748
FPS: 29
Writing Frame to: ../data/frames/train/outdoors/iPJ86uecLD0_000675.j

...

Writing Frame to: ../data/frames/train/outdoors/C-rF9sg1rPc_001366.jpg
Writing Frame to: ../data/frames/train/outdoors/C-rF9sg1rPc_002049.jpg
Writing Frame to: ../data/frames/train/outdoors/C-rF9sg1rPc_002732.jpg
Writing Frame to: ../data/frames/train/outdoors/C-rF9sg1rPc_003415.jpg
Writing Frame to: ../data/frames/train/outdoors/C-rF9sg1rPc_004098.jpg
Writing Frame to: ../data/frames/train/outdoors/C-rF9sg1rPc_004781.jpg
Writing Frame to: ../data/frames/train/outdoors/C-rF9sg1rPc_005464.jpg
Writing Frame to: ../data/frames/train/outdoors/C-rF9sg1rPc_006147.jpg
[youtube] iuP0yvtZFz0: Downloading webpage
[youtube] iuP0yvtZFz0: Downloading video info webpage
Total Frame Count: 4594
FPS: 29
Writing Frame to: ../data/frames/train/indoors/iuP0yvtZFz0_000460.jpg
Writing Frame to: ../data/frames/train/indoors/iuP0yvtZFz0_000920.jpg
Writing Frame to: ../data/frames/train/indoors/iuP0yvtZFz0_001380.jpg
Writing Frame to: ../data/frames/train/indoors/iuP0yvtZFz0_001840.jpg
Writing Frame to: ../da

ERROR: This video contains content from SME, who has blocked it in your country on copyright grounds.
Sorry about that.


Video not available, DownloadError
[youtube] 67I6gKp4XZY: Downloading webpage
[youtube] 67I6gKp4XZY: Downloading video info webpage
[youtube] 67I6gKp4XZY: Downloading MPD manifest
[youtube] mxOs-VwcK0E: Downloading webpage
[youtube] mxOs-VwcK0E: Downloading video info webpage
[youtube] mxOs-VwcK0E: Downloading MPD manifest
Total Frame Count: 11697
FPS: 30
Writing Frame to: ../data/frames/train/indoors/mxOs-VwcK0E_001170.jpg
Writing Frame to: ../data/frames/train/indoors/mxOs-VwcK0E_002340.jpg
Writing Frame to: ../data/frames/train/indoors/mxOs-VwcK0E_003510.jpg
Writing Frame to: ../data/frames/train/indoors/mxOs-VwcK0E_004680.jpg
Writing Frame to: ../data/frames/train/indoors/mxOs-VwcK0E_005850.jpg
Writing Frame to: ../data/frames/train/indoors/mxOs-VwcK0E_007020.jpg
Writing Frame to: ../data/frames/train/indoors/mxOs-VwcK0E_008190.jpg
Writing Frame to: ../data/frames/train/indoors/mxOs-VwcK0E_009360.jpg
Writing Frame to: ../data/frames/train/indoors/mxOs-VwcK0E_010530.jpg
[youtube] Vf

Writing Frame to: ../data/frames/train/outdoors/w1QdKfuZl1U_004688.jpg
Writing Frame to: ../data/frames/train/outdoors/w1QdKfuZl1U_005274.jpg
[youtube] T9rnNR9_DY8: Downloading webpage
[youtube] T9rnNR9_DY8: Downloading video info webpage
Total Frame Count: 5999
FPS: 29
Writing Frame to: ../data/frames/train/indoors/T9rnNR9_DY8_000600.jpg
Writing Frame to: ../data/frames/train/indoors/T9rnNR9_DY8_001200.jpg
Writing Frame to: ../data/frames/train/indoors/T9rnNR9_DY8_001800.jpg
Writing Frame to: ../data/frames/train/indoors/T9rnNR9_DY8_002400.jpg
Writing Frame to: ../data/frames/train/indoors/T9rnNR9_DY8_003000.jpg
Writing Frame to: ../data/frames/train/indoors/T9rnNR9_DY8_003600.jpg
Writing Frame to: ../data/frames/train/indoors/T9rnNR9_DY8_004200.jpg
Writing Frame to: ../data/frames/train/indoors/T9rnNR9_DY8_004800.jpg
Writing Frame to: ../data/frames/train/indoors/T9rnNR9_DY8_005400.jpg
[youtube] BR7LiNAbH8A: Downloading webpage
[youtube] BR7LiNAbH8A: Downloading video info webpage
[y

Total Frame Count: 7222
FPS: 25
Writing Frame to: ../data/frames/train/outdoors/BXG-3R6qQhk_000723.jpg
Writing Frame to: ../data/frames/train/outdoors/BXG-3R6qQhk_001446.jpg
Writing Frame to: ../data/frames/train/outdoors/BXG-3R6qQhk_002169.jpg
Writing Frame to: ../data/frames/train/outdoors/BXG-3R6qQhk_002892.jpg
Writing Frame to: ../data/frames/train/outdoors/BXG-3R6qQhk_003615.jpg
Writing Frame to: ../data/frames/train/outdoors/BXG-3R6qQhk_004338.jpg
Writing Frame to: ../data/frames/train/outdoors/BXG-3R6qQhk_005061.jpg
Writing Frame to: ../data/frames/train/outdoors/BXG-3R6qQhk_005784.jpg
Writing Frame to: ../data/frames/train/outdoors/BXG-3R6qQhk_006507.jpg
[youtube] LVzLTjATkIY: Downloading webpage
[youtube] LVzLTjATkIY: Downloading video info webpage
Total Frame Count: 3376
FPS: 25
Writing Frame to: ../data/frames/train/outdoors/LVzLTjATkIY_000338.jpg
Writing Frame to: ../data/frames/train/outdoors/LVzLTjATkIY_000676.jpg
Writing Frame to: ../data/frames/train/outdoors/LVzLTjATkI

In [46]:
for _, row in test_df.iterrows():
    output_dir = f"../data/frames/test/{row['target']}"
    get_frames(row['yt_id'], output_dir, 10, '240p')

[youtube] l-bfbZoUJ7o: Downloading webpage
[youtube] l-bfbZoUJ7o: Downloading video info webpage
Total Frame Count: 5309
FPS: 25
Writing Frame to: ../data/frames/test/indoors/l-bfbZoUJ7o_000531.jpg
Writing Frame to: ../data/frames/test/indoors/l-bfbZoUJ7o_001062.jpg
Writing Frame to: ../data/frames/test/indoors/l-bfbZoUJ7o_001593.jpg
Writing Frame to: ../data/frames/test/indoors/l-bfbZoUJ7o_002124.jpg
Writing Frame to: ../data/frames/test/indoors/l-bfbZoUJ7o_002655.jpg
Writing Frame to: ../data/frames/test/indoors/l-bfbZoUJ7o_003186.jpg
Writing Frame to: ../data/frames/test/indoors/l-bfbZoUJ7o_003717.jpg
Writing Frame to: ../data/frames/test/indoors/l-bfbZoUJ7o_004248.jpg
Writing Frame to: ../data/frames/test/indoors/l-bfbZoUJ7o_004779.jpg
[youtube] qs8mqzNhRxI: Downloading webpage
[youtube] qs8mqzNhRxI: Downloading video info webpage
Total Frame Count: 7687
FPS: 25
Writing Frame to: ../data/frames/test/outdoors/qs8mqzNhRxI_000769.jpg
Writing Frame to: ../data/frames/test/outdoors/qs8m

Writing Frame to: ../data/frames/test/indoors/EX9uh_SCXuI_003143.jpg
Writing Frame to: ../data/frames/test/indoors/EX9uh_SCXuI_003592.jpg
Writing Frame to: ../data/frames/test/indoors/EX9uh_SCXuI_004041.jpg
[youtube] iQMFGUAJ24A: Downloading webpage
[youtube] iQMFGUAJ24A: Downloading video info webpage
[youtube] iQMFGUAJ24A: Downloading MPD manifest
[youtube] YMcgLrTLh08: Downloading webpage
[youtube] YMcgLrTLh08: Downloading video info webpage
Total Frame Count: 5637
FPS: 25
Writing Frame to: ../data/frames/test/outdoors/YMcgLrTLh08_000564.jpg
Writing Frame to: ../data/frames/test/outdoors/YMcgLrTLh08_001128.jpg
Writing Frame to: ../data/frames/test/outdoors/YMcgLrTLh08_001692.jpg
Writing Frame to: ../data/frames/test/outdoors/YMcgLrTLh08_002256.jpg
Writing Frame to: ../data/frames/test/outdoors/YMcgLrTLh08_002820.jpg
Writing Frame to: ../data/frames/test/outdoors/YMcgLrTLh08_003384.jpg
Writing Frame to: ../data/frames/test/outdoors/YMcgLrTLh08_003948.jpg
Writing Frame to: ../data/fra

[youtube] qzayoabkDJU: Downloading MPD manifest
[youtube] tTOqfRcCJi4: Downloading webpage
[youtube] tTOqfRcCJi4: Downloading video info webpage
[youtube] tTOqfRcCJi4: Downloading MPD manifest
[youtube] n6FNjfDkLDQ: Downloading webpage
[youtube] n6FNjfDkLDQ: Downloading video info webpage
[youtube] n6FNjfDkLDQ: Downloading MPD manifest
[youtube] VRWdfIR8UrU: Downloading webpage
[youtube] VRWdfIR8UrU: Downloading video info webpage
Total Frame Count: 5489
FPS: 29
Writing Frame to: ../data/frames/test/outdoors/VRWdfIR8UrU_000549.jpg
Writing Frame to: ../data/frames/test/outdoors/VRWdfIR8UrU_001098.jpg
Writing Frame to: ../data/frames/test/outdoors/VRWdfIR8UrU_001647.jpg
Writing Frame to: ../data/frames/test/outdoors/VRWdfIR8UrU_002196.jpg
Writing Frame to: ../data/frames/test/outdoors/VRWdfIR8UrU_002745.jpg
Writing Frame to: ../data/frames/test/outdoors/VRWdfIR8UrU_003294.jpg
Writing Frame to: ../data/frames/test/outdoors/VRWdfIR8UrU_003843.jpg
Writing Frame to: ../data/frames/test/outdo

[youtube] bImL6XyWotA: Downloading video info webpage
Total Frame Count: 13828
FPS: 29
Writing Frame to: ../data/frames/test/indoors/bImL6XyWotA_001383.jpg
Writing Frame to: ../data/frames/test/indoors/bImL6XyWotA_002766.jpg
Writing Frame to: ../data/frames/test/indoors/bImL6XyWotA_004149.jpg
Writing Frame to: ../data/frames/test/indoors/bImL6XyWotA_005532.jpg
Writing Frame to: ../data/frames/test/indoors/bImL6XyWotA_006915.jpg
Writing Frame to: ../data/frames/test/indoors/bImL6XyWotA_008298.jpg
Writing Frame to: ../data/frames/test/indoors/bImL6XyWotA_009681.jpg
Writing Frame to: ../data/frames/test/indoors/bImL6XyWotA_011064.jpg
Writing Frame to: ../data/frames/test/indoors/bImL6XyWotA_012447.jpg
[youtube] W0DpVh2Q2dQ: Downloading webpage
[youtube] W0DpVh2Q2dQ: Downloading video info webpage
Total Frame Count: 5577
FPS: 29
Writing Frame to: ../data/frames/test/indoors/W0DpVh2Q2dQ_000558.jpg
Writing Frame to: ../data/frames/test/indoors/W0DpVh2Q2dQ_001116.jpg
Writing Frame to: ../data

Writing Frame to: ../data/frames/test/outdoors/k1fHi_Nghso_000831.jpg
Writing Frame to: ../data/frames/test/outdoors/k1fHi_Nghso_001108.jpg
Writing Frame to: ../data/frames/test/outdoors/k1fHi_Nghso_001385.jpg
Writing Frame to: ../data/frames/test/outdoors/k1fHi_Nghso_001662.jpg
Writing Frame to: ../data/frames/test/outdoors/k1fHi_Nghso_001939.jpg
Writing Frame to: ../data/frames/test/outdoors/k1fHi_Nghso_002216.jpg
Writing Frame to: ../data/frames/test/outdoors/k1fHi_Nghso_002493.jpg
Writing Frame to: ../data/frames/test/outdoors/k1fHi_Nghso_002770.jpg
[youtube] dwq51s2b6CA: Downloading webpage
[youtube] dwq51s2b6CA: Downloading video info webpage
[youtube] dwq51s2b6CA: Downloading MPD manifest
[youtube] HsrtI76aXig: Downloading webpage
[youtube] HsrtI76aXig: Downloading video info webpage
Total Frame Count: 7041
FPS: 25
Writing Frame to: ../data/frames/test/indoors/HsrtI76aXig_000705.jpg
Writing Frame to: ../data/frames/test/indoors/HsrtI76aXig_001410.jpg
Writing Frame to: ../data/fr

...

Writing Frame to: ../data/frames/test/outdoors/Yx2YwYsi5tI_002430.jpg
Writing Frame to: ../data/frames/test/outdoors/Yx2YwYsi5tI_002835.jpg
Writing Frame to: ../data/frames/test/outdoors/Yx2YwYsi5tI_003240.jpg
Writing Frame to: ../data/frames/test/outdoors/Yx2YwYsi5tI_003645.jpg
[youtube] eyJk5BaWeu8: Downloading webpage
[youtube] eyJk5BaWeu8: Downloading video info webpage
Total Frame Count: 8259
FPS: 29
Writing Frame to: ../data/frames/test/indoors/eyJk5BaWeu8_000826.jpg
Writing Frame to: ../data/frames/test/indoors/eyJk5BaWeu8_001652.jpg
Writing Frame to: ../data/frames/test/indoors/eyJk5BaWeu8_002478.jpg
Writing Frame to: ../data/frames/test/indoors/eyJk5BaWeu8_003304.jpg
Writing Frame to: ../data/frames/test/indoors/eyJk5BaWeu8_004130.jpg
Writing Frame to: ../data/frames/test/indoors/eyJk5BaWeu8_004956.jpg
Writing Frame to: ../data/frames/test/indoors/eyJk5BaWeu8_005782.jpg
Writing Frame to: ../data/frames/test/indoors/eyJk5BaWeu8_006608.jpg
Writing Frame to: ../data/frames/test/i

Writing Frame to: ../data/frames/test/indoors/Ht1tKcJfnuM_009837.jpg
[youtube] n__CyJk9rAk: Downloading webpage
[youtube] n__CyJk9rAk: Downloading video info webpage
Total Frame Count: 9336
FPS: 24
Writing Frame to: ../data/frames/test/indoors/n__CyJk9rAk_000934.jpg
Writing Frame to: ../data/frames/test/indoors/n__CyJk9rAk_001868.jpg
Writing Frame to: ../data/frames/test/indoors/n__CyJk9rAk_002802.jpg
Writing Frame to: ../data/frames/test/indoors/n__CyJk9rAk_003736.jpg
Writing Frame to: ../data/frames/test/indoors/n__CyJk9rAk_004670.jpg
Writing Frame to: ../data/frames/test/indoors/n__CyJk9rAk_005604.jpg
Writing Frame to: ../data/frames/test/indoors/n__CyJk9rAk_006538.jpg
Writing Frame to: ../data/frames/test/indoors/n__CyJk9rAk_007472.jpg
Writing Frame to: ../data/frames/test/indoors/n__CyJk9rAk_008406.jpg
[youtube] MUm6brG1vfs: Downloading webpage
[youtube] MUm6brG1vfs: Downloading video info webpage
Total Frame Count: 7002
FPS: 29
Writing Frame to: ../data/frames/test/outdoors/MUm6b

Writing Frame to: ../data/frames/test/outdoors/nqsgaTlmMGU_000754.jpg
Writing Frame to: ../data/frames/test/outdoors/nqsgaTlmMGU_001131.jpg
Writing Frame to: ../data/frames/test/outdoors/nqsgaTlmMGU_001508.jpg
Writing Frame to: ../data/frames/test/outdoors/nqsgaTlmMGU_001885.jpg
Writing Frame to: ../data/frames/test/outdoors/nqsgaTlmMGU_002262.jpg
Writing Frame to: ../data/frames/test/outdoors/nqsgaTlmMGU_002639.jpg
Writing Frame to: ../data/frames/test/outdoors/nqsgaTlmMGU_003016.jpg
Writing Frame to: ../data/frames/test/outdoors/nqsgaTlmMGU_003393.jpg
[youtube] goG9l_IVxOM: Downloading webpage
[youtube] goG9l_IVxOM: Downloading video info webpage
Total Frame Count: 7030
FPS: 25
Writing Frame to: ../data/frames/test/outdoors/goG9l_IVxOM_000704.jpg
Writing Frame to: ../data/frames/test/outdoors/goG9l_IVxOM_001408.jpg
Writing Frame to: ../data/frames/test/outdoors/goG9l_IVxOM_002112.jpg
Writing Frame to: ../data/frames/test/outdoors/goG9l_IVxOM_002816.jpg
Writing Frame to: ../data/frame

[youtube] -NtQslRYxDw: Downloading video info webpage
Total Frame Count: 3272
FPS: 25
Writing Frame to: ../data/frames/test/indoors/-NtQslRYxDw_000328.jpg
Writing Frame to: ../data/frames/test/indoors/-NtQslRYxDw_000656.jpg
Writing Frame to: ../data/frames/test/indoors/-NtQslRYxDw_000984.jpg
Writing Frame to: ../data/frames/test/indoors/-NtQslRYxDw_001312.jpg
Writing Frame to: ../data/frames/test/indoors/-NtQslRYxDw_001640.jpg
Writing Frame to: ../data/frames/test/indoors/-NtQslRYxDw_001968.jpg
Writing Frame to: ../data/frames/test/indoors/-NtQslRYxDw_002296.jpg
Writing Frame to: ../data/frames/test/indoors/-NtQslRYxDw_002624.jpg
Writing Frame to: ../data/frames/test/indoors/-NtQslRYxDw_002952.jpg
[youtube] v3k1nUk7Wyg: Downloading webpage
[youtube] v3k1nUk7Wyg: Downloading video info webpage
Total Frame Count: 5158
FPS: 30
Writing Frame to: ../data/frames/test/outdoors/v3k1nUk7Wyg_000516.jpg
Writing Frame to: ../data/frames/test/outdoors/v3k1nUk7Wyg_001032.jpg
Writing Frame to: ../dat

Writing Frame to: ../data/frames/test/outdoors/7orc7oPU8Z4_003060.jpg
Writing Frame to: ../data/frames/test/outdoors/7orc7oPU8Z4_003672.jpg
Writing Frame to: ../data/frames/test/outdoors/7orc7oPU8Z4_004284.jpg
Writing Frame to: ../data/frames/test/outdoors/7orc7oPU8Z4_004896.jpg
Writing Frame to: ../data/frames/test/outdoors/7orc7oPU8Z4_005508.jpg
[youtube] vPysHxkJX7k: Downloading webpage
[youtube] vPysHxkJX7k: Downloading video info webpage
Total Frame Count: 10395
FPS: 30
Writing Frame to: ../data/frames/test/outdoors/vPysHxkJX7k_001040.jpg
Writing Frame to: ../data/frames/test/outdoors/vPysHxkJX7k_002080.jpg
Writing Frame to: ../data/frames/test/outdoors/vPysHxkJX7k_003120.jpg
Writing Frame to: ../data/frames/test/outdoors/vPysHxkJX7k_004160.jpg
Writing Frame to: ../data/frames/test/outdoors/vPysHxkJX7k_005200.jpg
Writing Frame to: ../data/frames/test/outdoors/vPysHxkJX7k_006240.jpg
Writing Frame to: ../data/frames/test/outdoors/vPysHxkJX7k_007280.jpg
Writing Frame to: ../data/fram